# Without Metadata

## Importing the libraries

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.stats.multitest import multipletests
import statsmodels.api as sm
import statsmodels.stats.multitest as smm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Importing the dataset

In [ ]:
clr_data = pd.read_excel('/content/drive/MyDrive/hmds Assignments/Assignment1_ClrTrans_Species.xlsx', index_col=0)
metadata = pd.read_excel('/content/drive/MyDrive/hmds Assignments/Assignment1_Metadata.xlsx', index_col=0)

## Filter out species detected in less than 5% of the samples

In [ ]:
detected_species = (clr_data > 0).sum(axis=1)
clr_data = clr_data.loc[detected_species[detected_species >= len(clr_data)*0.05].index]

In [ ]:
clr_data

,Bifidobacterium_dentium,Butyricimonas_synergistica,Parvibacter_caecicola,Clostridium_sartagoforme,Ruminococcus_gauvreauii,Bacteroides_stercoris,Bacteroides_plebeius,Streptococcus_parasanguinis,Enterococcus_rivorum,Clostridium_methylpentosum,...,Anaerotruncus_colihominis,Lactobacillus_sakei,Eubacterium_xylanophilum,Slackia_isoflavoniconvertens,Alistipes_massiliensis,Veillonella_atypica,Enterococcus_casseliflavus,Olsenella_profusa,Lactobacillus_reuteri,Peptococcus_niger
AIIDB0330,2.484907,2.197225,0.000000,0.000000,3.332205,7.753624,0.000000,4.442651,0.000000,4.919981,...,7.163947,0.000000,2.890372,5.525453,5.170484,1.098612,0.000000,1.609438,4.418841,0.000000
AIIDM0042,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
AIIDM0318,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.258097,...,0.000000,0.000000,0.000000,0.000000,0.693147,3.761200,0.000000,0.000000,0.000000,0.000000
AIIDV1015,0.693147,0.000000,0.000000,0.693147,1.609438,0.000000,0.000000,0.000000,0.000000,0.693147,...,0.000000,0.000000,1.791759,3.583519,0.693147,0.000000,0.000000,0.693147,0.000000,0.000000
AIIDV1406,0.000000,0.000000,0.000000,0.000000,2.564949,0.000000,0.000000,0.000000,0.000000,2.079442,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AIIDM0040,1.609438,0.000000,0.000000,0.000000,1.609438,0.000000,0.000000,0.693147,0.000000,0.000000,...,0.000000,0.000000,0.000000,2.944439,0.000000,2.833213,0.000000,0.000000,0.000000,0.000000
COVIRL-201-015,7.774856,0.000000,0.000000,3.555348,1.098612,0.000000,6.023448,4.025352,0.693147,6.431331,...,5.342334,0.000000,0.000000,0.000000,0.000000,0.000000,2.197225,0.000000,0.000000,0.000000
AIIDV1085,0.000000,0.000000,0.000000,0.000000,0.000000,1.386294,0.000000,0.000000,0.000000,1.609438,...,1.386294,0.000000,0.000000,2.833213,0.000000,0.000000,0.000000,0.693147,0.000000,4.779123
AIIDM0020,0.000000,0.000000,0.000000,0.000000,3.044522,0.000000,0.000000,0.000000,0.000000,2.995732,...,2.302585,1.098612,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## Convert Covid-19 severity and Gender to numeric encoding

In [ ]:
severity_mapping = {'mild': 1, 'moderate': 2, 'critical_severe': 3}
gender_mapping = {'F':0,'M':1}
metadata['WHO_severity'] = metadata['WHO_severity'].map(severity_mapping)
metadata['Sex'] = metadata['Sex'].map(gender_mapping)

In [ ]:
metadata

,Sex,Age,BMI,HTN,Diabetes,Respiratory_disease,Heart_disease,Renal_Disease,Liver_Disease,Obesity,Malignancy,Immunosuppressive_Disease,Neurological_disease,Metabolic_Disease,Cardiovascular_Disease,comorbidities_total,WHO_severity
AIIDB0330,1,63,48.995023,n,y,n,y,y,n,y,n,n,n,y,y,6,3
AIIDM0042,1,63,30.322325,y,n,n,n,n,n,n,n,n,n,y,y,3,2
AIIDM0318,1,41,30.322325,n,n,n,n,n,n,n,n,n,n,n,n,0,3
AIIDV1015,0,42,57.795561,n,n,n,n,n,n,y,n,n,n,y,n,2,1
AIIDV1406,0,85,23.728191,y,y,y,y,y,n,n,y,n,n,y,y,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AIIDM0040,0,43,30.322325,n,n,n,n,n,n,n,n,n,n,n,n,0,2
COVIRL-201-015,0,87,25.562130,n,n,y,y,n,n,n,n,n,n,n,y,3,3
AIIDV1085,1,78,26.543210,n,y,n,y,n,n,n,y,n,n,y,y,5,3
AIIDM0020,1,74,30.322325,y,n,n,y,n,n,n,n,n,n,y,y,4,1


## Perform linear regression for each species

In [ ]:
results = []
for species in clr_data.columns:
    x = clr_data[species]
    y = metadata['WHO_severity']
    model = sm.OLS(y, sm.add_constant(x)).fit()
    results.append((species, model.pvalues[1]))

## Correct for multiple testing using the Benjamini-Hochberg method

In [ ]:
p_values = [i[1] for i in results]
reject, pvalues, _, _ = multipletests(p_values, method='fdr_bh')

# extract the significant species
sig_species = [r[0] for i, r in enumerate(results) if pvalues[i] <= 0.1]


## Significant species

In [ ]:
# Print significant species
print('Significant species after adjustment (FDR <= 0.1):', len(sig_species))

Significant species after adjustment (FDR <= 0.1): 83


In [ ]:
[(r[0],r[1]) for i, r in enumerate(results) if pvalues[i] <= 0.1]

[('Clostridium_sartagoforme', 0.001814317037460714),
 ('Bacteroides_plebeius', 0.0025090452648332882),
 ('Clostridium_methylpentosum', 0.0003101867914927888),
 ('Escherichia.Shigella_flexneri', 0.030099378162692807),
 ('Subdoligranulum_variabile', 0.011951201287901413),
 ('Clostridium_disporicum', 0.0003706793559823975),
 ('Finegoldia_magna', 0.0016203824930970496),
 ('Bacteroides_acidifaciens', 0.00015912151855799437),
 ('Atopobium_parvulum', 0.006632599675692202),
 ('Blautia_schinkii', 0.007120391799397053),
 ('Clostridium_aminobutyricum', 0.01050132855846613),
 ('Enterococcus_mundtii', 0.011026385161287353),
 ('Anaerostipes_butyraticus', 0.009269569552335741),
 ('Acidaminococcus_intestini', 0.01645526176783011),
 ('Methanobrevibacter_smithii', 4.0535270913807776e-05),
 ('Eubacterium_siraeum', 0.01510024826869974),
 ('Solobacterium_moorei', 0.0012923564702762627),
 ('Anaerosporobacter_mobilis', 0.006673868521490599),
 ('Blautia_stercoris', 0.008097619254980874),
 ('Melissococcus_plut

In [ ]:
sig_species

['Clostridium_sartagoforme',
 'Bacteroides_plebeius',
 'Clostridium_methylpentosum',
 'Escherichia.Shigella_flexneri',
 'Subdoligranulum_variabile',
 'Clostridium_disporicum',
 'Finegoldia_magna',
 'Bacteroides_acidifaciens',
 'Atopobium_parvulum',
 'Blautia_schinkii',
 'Clostridium_aminobutyricum',
 'Enterococcus_mundtii',
 'Anaerostipes_butyraticus',
 'Acidaminococcus_intestini',
 'Methanobrevibacter_smithii',
 'Eubacterium_siraeum',
 'Solobacterium_moorei',
 'Anaerosporobacter_mobilis',
 'Blautia_stercoris',
 'Melissococcus_plutonius',
 'Clostridium_bifermentans',
 'Christensenella_minuta',
 'Streptococcus_sanguinis',
 'Actinomyces_graevenitzii',
 'Pseudoflavonifractor_capillosus',
 'Coprobacillus_cateniformis',
 'Lactobacillus_fermentum',
 'Clostridium_viride',
 'Eubacterium_contortum',
 'Actinomyces_odontolyticus',
 'Actinomyces_lingnae',
 'Clostridium_irregulare',
 'Acetanaerobacterium_elongatum',
 'Clostridium_aldenense',
 'Blautia_luti',
 'Bacteroides_ovatus',
 'Anaerofilum_pen

# With Including Metadata

## Importing the dataset

In [ ]:
clr_data_df = pd.read_excel('/content/drive/MyDrive/hmds Assignments/Assignment1_ClrTrans_Species.xlsx', index_col=0)
metadata_df = pd.read_excel('/content/drive/MyDrive/hmds Assignments/Assignment1_Metadata.xlsx', index_col=0)

## Filter out species detected in less than 5% of the samples

In [ ]:
detected_species = (clr_data_df > 0).sum(axis=1)
clr_data_df = clr_data_df.loc[detected_species[detected_species >= len(clr_data_df)*0.05].index]

In [ ]:
clr_data_df

,Bifidobacterium_dentium,Butyricimonas_synergistica,Parvibacter_caecicola,Clostridium_sartagoforme,Ruminococcus_gauvreauii,Bacteroides_stercoris,Bacteroides_plebeius,Streptococcus_parasanguinis,Enterococcus_rivorum,Clostridium_methylpentosum,...,Anaerotruncus_colihominis,Lactobacillus_sakei,Eubacterium_xylanophilum,Slackia_isoflavoniconvertens,Alistipes_massiliensis,Veillonella_atypica,Enterococcus_casseliflavus,Olsenella_profusa,Lactobacillus_reuteri,Peptococcus_niger
AIIDB0330,2.484907,2.197225,0.000000,0.000000,3.332205,7.753624,0.000000,4.442651,0.000000,4.919981,...,7.163947,0.000000,2.890372,5.525453,5.170484,1.098612,0.000000,1.609438,4.418841,0.000000
AIIDM0042,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
AIIDM0318,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.258097,...,0.000000,0.000000,0.000000,0.000000,0.693147,3.761200,0.000000,0.000000,0.000000,0.000000
AIIDV1015,0.693147,0.000000,0.000000,0.693147,1.609438,0.000000,0.000000,0.000000,0.000000,0.693147,...,0.000000,0.000000,1.791759,3.583519,0.693147,0.000000,0.000000,0.693147,0.000000,0.000000
AIIDV1406,0.000000,0.000000,0.000000,0.000000,2.564949,0.000000,0.000000,0.000000,0.000000,2.079442,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AIIDM0040,1.609438,0.000000,0.000000,0.000000,1.609438,0.000000,0.000000,0.693147,0.000000,0.000000,...,0.000000,0.000000,0.000000,2.944439,0.000000,2.833213,0.000000,0.000000,0.000000,0.000000
COVIRL-201-015,7.774856,0.000000,0.000000,3.555348,1.098612,0.000000,6.023448,4.025352,0.693147,6.431331,...,5.342334,0.000000,0.000000,0.000000,0.000000,0.000000,2.197225,0.000000,0.000000,0.000000
AIIDV1085,0.000000,0.000000,0.000000,0.000000,0.000000,1.386294,0.000000,0.000000,0.000000,1.609438,...,1.386294,0.000000,0.000000,2.833213,0.000000,0.000000,0.000000,0.693147,0.000000,4.779123
AIIDM0020,0.000000,0.000000,0.000000,0.000000,3.044522,0.000000,0.000000,0.000000,0.000000,2.995732,...,2.302585,1.098612,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## Convert Covid-19 severity and Gender to numeric encoding

In [ ]:
severity_mapping = {'mild': 1, 'moderate': 2, 'critical_severe': 3}
gender_mapping = {'F':0,'M':1}
metadata_df['WHO_severity'] = metadata_df['WHO_severity'].map(severity_mapping)
metadata_df['Sex'] = metadata_df['Sex'].map(gender_mapping)

In [ ]:
metadata_df

,Sex,Age,BMI,HTN,Diabetes,Respiratory_disease,Heart_disease,Renal_Disease,Liver_Disease,Obesity,Malignancy,Immunosuppressive_Disease,Neurological_disease,Metabolic_Disease,Cardiovascular_Disease,comorbidities_total,WHO_severity
AIIDB0330,1,63,48.995023,n,y,n,y,y,n,y,n,n,n,y,y,6,3
AIIDM0042,1,63,30.322325,y,n,n,n,n,n,n,n,n,n,y,y,3,2
AIIDM0318,1,41,30.322325,n,n,n,n,n,n,n,n,n,n,n,n,0,3
AIIDV1015,0,42,57.795561,n,n,n,n,n,n,y,n,n,n,y,n,2,1
AIIDV1406,0,85,23.728191,y,y,y,y,y,n,n,y,n,n,y,y,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AIIDM0040,0,43,30.322325,n,n,n,n,n,n,n,n,n,n,n,n,0,2
COVIRL-201-015,0,87,25.562130,n,n,y,y,n,n,n,n,n,n,n,y,3,3
AIIDV1085,1,78,26.543210,n,y,n,y,n,n,n,y,n,n,y,y,5,3
AIIDM0020,1,74,30.322325,y,n,n,y,n,n,n,n,n,n,y,y,4,1


## Perform multi linear regression for each species

In [ ]:
result = []
covariates = ['Age', 'BMI', 'Sex', 'comorbidities_total']
X = metadata_df[covariates]
for species in clr_data_df.columns:
    x = clr_data_df[species]
    y = metadata_df['WHO_severity']
    df = pd.concat([clr_data[species],X],axis=1)
    x = sm.add_constant(df)
    model = sm.OLS(y, x).fit()
    # p_values = model.summary2().tables[1]['P>|t|']
    result.append((species, model.pvalues[1]))

## Correct for multiple testing using the Benjamini-Hochberg method

In [ ]:
pvalues = [r[1] for r in result]
reject, p_values_fdr, _, _ = multipletests(pvalues, alpha=0.1, method='fdr_bh')

# extract the significant species
new_sig_species = [r[0] for i, r in enumerate(results) if p_values_fdr[i] <= 0.1]

## Significant species

In [ ]:
# Print significant species
print('Significant species after adjustment (FDR <= 0.1):', len(new_sig_species))

Significant species after adjustment (FDR <= 0.1): 84


In [ ]:
[(r[0],r[1]) for i, r in enumerate(results) if p_values_fdr[i] <= 0.1]

[('Clostridium_sartagoforme', 0.001814317037460714),
 ('Bacteroides_plebeius', 0.0025090452648332882),
 ('Clostridium_methylpentosum', 0.0003101867914927888),
 ('Escherichia.Shigella_flexneri', 0.030099378162692807),
 ('Subdoligranulum_variabile', 0.011951201287901413),
 ('Clostridium_disporicum', 0.0003706793559823975),
 ('Finegoldia_magna', 0.0016203824930970496),
 ('Bacteroides_acidifaciens', 0.00015912151855799437),
 ('Atopobium_parvulum', 0.006632599675692202),
 ('Streptococcus_suis', 0.04801078046036608),
 ('Clostridium_scindens', 0.03257525239215355),
 ('Blautia_schinkii', 0.007120391799397053),
 ('Clostridium_aminobutyricum', 0.01050132855846613),
 ('Enterococcus_mundtii', 0.011026385161287353),
 ('Anaerostipes_butyraticus', 0.009269569552335741),
 ('Acidaminococcus_intestini', 0.01645526176783011),
 ('Lactobacillus_delbrueckii', 0.06355245971461616),
 ('Methanobrevibacter_smithii', 4.0535270913807776e-05),
 ('Eubacterium_siraeum', 0.01510024826869974),
 ('Solobacterium_moorei'

In [ ]:
new_sig_species

['Clostridium_sartagoforme',
 'Bacteroides_plebeius',
 'Clostridium_methylpentosum',
 'Escherichia.Shigella_flexneri',
 'Subdoligranulum_variabile',
 'Clostridium_disporicum',
 'Finegoldia_magna',
 'Bacteroides_acidifaciens',
 'Atopobium_parvulum',
 'Streptococcus_suis',
 'Clostridium_scindens',
 'Blautia_schinkii',
 'Clostridium_aminobutyricum',
 'Enterococcus_mundtii',
 'Anaerostipes_butyraticus',
 'Acidaminococcus_intestini',
 'Lactobacillus_delbrueckii',
 'Methanobrevibacter_smithii',
 'Eubacterium_siraeum',
 'Solobacterium_moorei',
 'Anaerosporobacter_mobilis',
 'Blautia_stercoris',
 'Melissococcus_plutonius',
 'Streptococcus_thermophilus',
 'Clostridium_bifermentans',
 'Christensenella_minuta',
 'Streptococcus_sanguinis',
 'Actinomyces_graevenitzii',
 'Pseudoflavonifractor_capillosus',
 'Coprobacillus_cateniformis',
 'Lactobacillus_fermentum',
 'Clostridium_viride',
 'Eubacterium_contortum',
 'Actinomyces_odontolyticus',
 'Actinomyces_lingnae',
 'Clostridium_irregulare',
 'Acetan